<img src="https://www.unir.net/wp-content/uploads/2019/11/Unir_2021_logo.svg" width="240" height="240" align="right"/>

<center><h1>Técnicas de Inteligencia Artificial</header1></center>
<left><h1>Actividad 1. Laboratorio: Árboles de decisión, reglas y ensemble learning</header1></left>

Presentado por: **Ramón Barrios Láscar**  <br>
Fecha: 01/12/2023

## Importación de librerias necesarias

In [2]:
#Para esta actividad se importarán las siguientes librerías:
import pandas as pd
import seaborn as sns
import sklearn as skl

## Cargar el Dataset

In [44]:
#Código para cargar el Dataset
url = 'https://raw.githubusercontent.com/oscar-unir/TIA/main/datasets/car/Laboratorio_dataset_car.csv'
df = pd.read_csv(url, sep=';', header='infer', encoding='utf-8')

<div class="alert alert-block alert-info"> <b>RBL: </b>Se carga el dataset directamente de la URL anotada en el notebook base, así se evita depender de el archivo local y su estructura.</div>

## Explique el problema a resolver. 
Descripción del problema. Tipo de problema (justifique). Variable objetivo, variables de entrada. Utilidad de su posible solución. Elementos adicionales que considere relevantes:

<div class="alert alert-block alert-info"> <b>RBL: </b>Según la especificaciones en el documento <code>visa04lab</code>, página <b>3</b>, se deberán usar dos algoritmos de clasificación, por lo que se trata entonces de un *problema de clasificación*. Antes de explorar detalladamente el dataset no se puede decir más, por lo que esto se tratará más adelante.</div>

## Caracterización del Dataset

Se incluye una descripción de los datos con:

>- Número de clases de la variable objetivo, indicando que representan dichas clases y el tipo de valor que toman.
>- Número de instancias en total.
>- Número de instancias pertenecientes a cada clase.
>- Número de atributos de entrada, su significado y tipo.
>- ¿Hay algún valor de atributo desconocido?

Se incorporá una descripción (EDA) del conjunto de datos utilizado. Se analiza el dataset proporcionando, se muestra al menos algunas de sus características mediante tablas y al menos algunas de ellas en modo gráfico (p.ej., histogramas, diagramas de dispersión, diagramas de cajas y bigotes, etc.)


In [45]:
print(f"El dataset está compuesto de %d instancias y %d atributos (filas×columnas)"%(df.shape[0],df.shape[1],))
print(f"Con los siguientes atributos:")
for col in df.columns:
    print(f"• {col}")

El dataset está compuesto de 1750 instancias y 7 atributos (filas×columnas)
Con los siguientes atributos:
• Buying
• Maintenance
• Doors
• Person
• lug_boot
• safety
• class


<div class="alert alert-block alert-info"> <b>RBL: </b>Como reporta el código, el dataset tiene <b>1749</b> instancias, con <b>atributos</b>:
<p><ul><li>Buying</li>
<li>Maintenance</li>
<li>Doors</li>
<li>Person</li>
<li>lug_boot</li>
<li>safety</li>
<li>class</li></ul>
</div>

In [62]:
for col in df.columns:
    print(f"Valores para la columna '{col}':")
    print(df.groupby([col])[col].count())
    print()
print("Tipos:")
df.dtypes

Valores para la columna 'Buying':
Buying
high     432
low      437
med      438
vhigh    443
Name: Buying, dtype: int64

Valores para la columna 'Maintenance':
Maintenance
high     432
low      447
med      434
vhigh    437
Name: Maintenance, dtype: int64

Valores para la columna 'Doors':
Doors
2        444
3        435
4        434
5more    437
Name: Doors, dtype: int64

Valores para la columna 'Person':
Person
2       578
4       587
more    585
Name: Person, dtype: int64

Valores para la columna 'lug_boot':
lug_boot
big      585
med      583
small    582
Name: lug_boot, dtype: int64

Valores para la columna 'safety':
safety
high    590
low     578
med     582
Name: safety, dtype: int64

Valores para la columna 'class':
class
acc       390
good       75
unacc    1215
vgood      70
Name: class, dtype: int64

Tipos:


Buying         object
Maintenance    object
Doors          object
Person         object
lug_boot       object
safety         object
class          object
dtype: object

<div class="alert alert-block alert-info"> <b>RBL: </b>
Se tienen las columnas ya mencionadas, que se detallan:<ul>
<li><b>Buying</b>: texto (categórica), con cuatro posibilidades, <b>low</b>, <code>437</code>; <b>med</b>, <code>438</code>; <b>high</b>, <code>432</code>; y <b>vhigh</b>, <code>443</code> <i>(parece ser el "precio de compra")</i></li>
<li><b>Maintenance</b>: texto (categórica), con cuatro posibilidades, <b>low</b>, <code>447</code>; <b>med</b>, <code>434</code>; <b>high</b>, <code>432</code>;  y <b>vhigh</b>, <code>437</code> <i>(parece ser el "costo de mantenimiento")</i></li>
<li><b>Doors</b>: texto (categórica), con cuatro posibilidades, <b>2</b>, <code>444</code>; <b>3</b>, <code>435</code>; <b>4</b>, <code>434</code>; y <b>5more</b>, <code>437</code> (esto es el número de puertas, con <i>5more</i> siendo "5 o más puertas")</li>
<li><b>Person</b>: texto (categórica), con tres posibilidades, <b>2</b>, <code>578</code>; <b>4</b>, <code>587</code>;  y <b>more</b>, <code>585</code>  (esto es el número de pasajeros, con <i>more</i> siendo "más de cuatro pasajeros")</li>
<li><b>lug_boot</b>: texto (categórica), con tres posibilidades, <b>small</b>, <code>582</code>; <b>med</b>, <code>583</code>; y <b>big</b>, <code>585</code>; (esto es el tamaño del baúl)</li>
<li><b>safety</b>: texto (categórica), con tres posibilidades, <b>low</b>, <code>578</code>; <b>med</b>, <code>582</code>; y <b>high</b>, <code>590</code> (esto es qué tan seguro se considera al vehículo)</li>
<li><b>class</b>: texto (categórica), con cuatro posibilidades, <b>unacc</b>, <code>1215</code>; <b>acc</b>, <code>390</code>; <b>good</b>, <code>75</code>; y <b>vgood</b>, <code>70</code> (esto parece ser "qué tan atractivo para la compra es", con <i>unacc</i> o "no", <i>acc</i> o "aceptable", <i>good</i> o "bien", y <i>vgood</i> o "muy")</li>
</ul>
<p>El texto <b>así</b> es uno de los valores del atributo y el número <code>así</code> es "cuántas instancias en esa clase"<p></div>

<div class="alert alert-block alert-info"> <b>RBL: </b><ul>
<li>Encontradas en <b>Buying</b> con valores: <code>432+437+438+443 = 1750</code> (todas)</li>
<li>Encontradas en <b>Maintenance</b> con valores: <code>447+434+432+437 = 1750</code> (todas)</li>
<li>Encontradas en <b>Doors</b> con valores: <code>444+435+434+437 = 1750</code> (todas)</li>
<li>Encontradas en <b>Person</b> con valores: <code>578+587+585 = 1750</code> (todas)</li>
<li>Encontradas en <b>lug_boot</b> con valores: <code>582+583+585 = 1750</code> (todas)</li>
<li>Encontradas en <b>safety</b> con valores: <code>578+582+590 = 1750</code> (todas)</li>
<li>Encontradas en <b>class</b> con valores: <code>1215+390+75+70 = 1750</code> (todas)</li>
</ul></p>
<p>Este es un dataset completamente limpio, sin nulos o "N/A"</div>

## Preprocesamiento del dataset. Transformaciones previas necesarias para la modelación

In [36]:
#Código que realice las transformaciones necesarias para poder realizar los procesos de modelación. Ej.One hot enconding

## División del dataset en datos de entrenamiento y datos de test 

In [37]:
#Código que realice la división en entrenamiento y test, de acuerdo con la estretgia de evluación planeada. Describa cuál es.

## Ajuste de los modelos de clasificación propuestos

Justifique la selección de las dos propuestas de modelación seleccionadas:

In [38]:
#Código de ajuste del modelo de clasificación 1

In [39]:
#Código de ajuste del modelo de clasificación 2

## Evaluación de cada modelo

Al menos incluya:

+ Instancias clasificadas correctamente
+ Instancias clasificadas incorrectamente
+ TP Rate
+ FP Rate
+ Matriz de confusión


In [40]:
#Código para mostrar la evaluación del modelo de clasificación 1

Construya un párrafo con los principales hallazgos.

In [41]:
#Código  para mostrarla evaluación del modelo de clasificación 2

Construya un párrafo con los principales hallazgos.

## Comparación del desempeño de modelos

In [42]:
#Código para mostrar la comparación de métricas de desempeño de las dos propuestas en tabla

In [43]:
#Código para mostrar la comparación de métricas de desempeño de las dos propuestas en gráfica

Construya un párrafo con los principales hallazgos.

## Discusión de los resultados obtenidos y argumentos sobre cómo se podrían mejorar de dichos resultados

Realice en este espacio todo el análsis de resultados final incluyendo: ventajas y desventajas de cada modelo propuesto, Resultados comparados. Conclusiones objetivas y significantes con base a las diferentes métricas escogidas. Recomendaciones de mejora de las propuestas: 